[Python Metaclasses](https://realpython.com/python-metaclasses/)

[What are metaclasses in Python?](https://stackoverflow.com/questions/100003/what-are-metaclasses-in-python)

> “Metaclasses are deeper magic than 99% of users should ever worry about.   
If you wonder whether you need them, you don’t  
(the people who actually need them know with certainty that they need them, and don’t need an explanation about why).”
> 
> — Tim Peters



In [55]:
for t in int, float, dict, list, tuple, object, type:
    print(type(t))

<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>


In [58]:
l = list([1,2,3])
print(l.__class__)
print(l.__class__.__class__)

<class 'list'>
<class 'type'>


## Defining a Class Dynamically

The built-in type() function, when passed one argument, returns the type of an object. For new-style classes, that is generally the same as the object’s __class__ attribute
```python
l = list([1,2,3])
print(l.__class__ == type(l))
print(l.__class__.__class__ == type(type(l)))
```
You can also call type() with three arguments—type(name, bases, dct):
* name: specifies the class name. This becomes the \__name__ attribute of the class.
* bases: specifies a tuple of the base classes from which the class inherits. This becomes the \__bases__ attribute of the class.
* dct: specifies a namespace dictionary containing definitions for the class body. This becomes the \__dict__ attribute of the class.

In [65]:
class A:
    a = 1

class B(A):
    b = 2

obj_b = B()
print(obj_b.a, obj_b.b)

1 2


## 深度思考
```python

def new(cls):
    x = object.__new__(cls)
    x.attr = 100
    return x
Foo.__new__ = new
# 通过类可以实例化对象。
# 我们可以定制实例化过程。
# 定制的核心在于修改__new__()函数。实例化最重要的一步是通过__new__()函数返回实例化的对象。
# 只要修改__new__()函数，就可以定制实例化过程，比如新添加一个属性。

class Meta(type):
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        x.attr = 100
        return x
# 通过type可以实例化类。
# 同理，我们也可以定制实例化类的过程。
# 定制的核心也在于修改__new__()函数。通过__new__()函数返回一个实例化的类。
```

```python
# 请对比
x = object.__new__(cls) #返回对象
x = super().__new__(cls, name, bases, dct) #返回类
```

In [48]:
class UpperAttrMetaclass(type):
    def __new__(cls, clsname, bases, attrs):
        uppercase_attrs = {
            attr if attr.startswith("__") else attr.upper(): v
            for attr, v in attrs.items()
        }

        return super().__new__(cls, clsname, bases, uppercase_attrs)

class Foo(object, metaclass=UpperAttrMetaclass):
    hello = 'world'

    def show(self):
        print('hello foo!')

foo = Foo()

foo.SHOW()

hello foo!


In [60]:
def upper_attr(future_class_name, future_class_parents, future_class_attrs):
    """
      Return a class object, with the list of its attribute turned
      into uppercase.
    """
    print('future_class_name: {}'.format(future_class_name))
    print('future_class_parents: {}'.format(future_class_parents))
    print('future_class_attrs: {}'.format(future_class_attrs))

    uppercase_attrs = {
        attr if attr.startswith("__") else attr.upper(): v
        for attr, v in future_class_attrs.items()
    }

    return type(future_class_name, future_class_parents, uppercase_attrs)

class Foo():
    pass

class Bar(Foo, metaclass=upper_attr): 
    def __init__(self, name):
        self.name = name
    hello = 'world'

    def show(self):
        print('hello bar!')

bar = Bar('thu')

bar.SHOW()

future_class_name: Bar
future_class_parents: (<class '__main__.Foo'>,)
future_class_attrs: {'__module__': '__main__', '__qualname__': 'Bar', '__init__': <function Bar.__init__ at 0x7f948885f550>, 'hello': 'world', 'show': <function Bar.show at 0x7f948885f430>}
hello bar!


In [18]:
bar.__dict__

{'name': 'thu'}

In [21]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        x.attr = 100
        return x

class Foo(metaclass=Meta):
    pass

Foo.attr

100

In [47]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        # 这些参数如何传入？
        # 原来如此！自动传入
        # name, bases, dct自动接收Foo类的相关信息
        print('cls: ', cls)
        print('name: ', name)
        print('bases: ', bases)

        print('dct before: ', dct)
        dct = {**dct, **dict(attr=101)}
        print('dct after: ', dct)

        print('super(): ', super())
        # super()究竟是对象和类？应该是对象
        res = super().__new__(cls, name, bases, dct)
        # res = type(name, bases, dct)
        print('res: ', res)

        return res

class Bar(object):
    pass

class Foo(Bar, metaclass=Meta):
    hello = 'world'

print(Foo.attr)
print(Foo.hello)

cls:  <class '__main__.Meta'>
name:  Foo
bases:  (<class '__main__.Bar'>,)
dct before:  {'__module__': '__main__', '__qualname__': 'Foo', 'hello': 'world'}
dct after:  {'__module__': '__main__', '__qualname__': 'Foo', 'hello': 'world', 'attr': 101}
super():  <super: <class 'Meta'>, <Meta object>>
res:  <class '__main__.Foo'>
101
world
